In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [8]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("What is weather in SF")
print(search_results)

[{'title': 'Weather in san francisco ca', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741415692, 'localtime': '2025-03-07 22:34'}, 'current': {'last_updated_epoch': 1741415400, 'last_updated': '2025-03-07 22:30', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 5.4, 'wind_kph': 8.6, 'wind_degree': 299, 'wind_dir': 'WNW', 'pressure_mb': 1023.0, 'pressure_in': 30.22, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 74, 'cloud': 0, 'feelslike_c': 10.2, 'feelslike_f': 50.3, 'windchill_c': 10.5, 'windchill_f': 50.9, 'heatindex_c': 10.8, 'heatindex_f': 51.4, 'dewpoint_c': 9.6, 'dewpoint_f': 49.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 10.4, 'gust_kph': 16.8}}", 'score': 0.7327484}, {'title': 'Weather Forecast and Conditions for San Francisco, CA', 'url': 'https://weather.com/weather/today/l/USCA0987:1:US', 'content': "Weather Forecast and Conditions for San Francisco, CA - The Weather Channel | Weather.com Weather Forecasts Today's Forecast for San Francisco, CA Weather Today in San Francisco, CA Northeast Snow Totals Likely To Surpass Past 2 Winters Combined ### Now 49° Chance of Rain2% ### 10 pm 48° Chance of Rain2% ### 11 pm 47° Chance of Rain2% ### 12 am 46° Chance of Rain2% Breathe Easier This Winter By Improving Indoor Air Quality Thank You Blithedale Canyon / Middle Ridge Neighbor 0d ago 9reactions6commentsMoreCoast cafe OPEN after our Winter Break!We are now back from our vacations and OPEN for our Winter Hours. Weather Data APIs We may use or share your data with our data vendors.", 'score': 0.5683445}]

# **Using Language Models**

In [ ]:
pip install -qU "langchain[groq]"

In [28]:
from secret_keys import groq_key
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = groq_key

from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [29]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [30]:
tools=[TavilySearchResults(max_results=2)]
model_with_tools = model.bind_tools(tools)

In [31]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Hi!'}, 'id': 'call_g1h3', 'type': 'tool_call'}]


In [32]:
response = model_with_tools.invoke([HumanMessage(content="Use the TavilySearchResults tool to answer the following question: What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': "what's the weather in SF"}, 'id': 'call_7c19', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

# **Using Agent**

In [33]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [36]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='f0b235ab-301b-4c0b-b2ed-f42561ea9726'),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 1902, 'total_tokens': 1905, 'completion_time': 0.0025, 'prompt_time': 0.240126013, 'queue_time': -0.291561198, 'total_time': 0.242626013}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-9c1f9911-404a-4609-ad0f-52091eeed656-0', usage_metadata={'input_tokens': 1902, 'output_tokens': 3, 'total_tokens': 1905})]

In [35]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='fe97a426-04ac-4530-921f-55438d6fd96b'),
 AIMessage(content="According to the latest forecast, it's currently sunny with a high of 65°F (18°C) and a low of 55°F (13°C) in San Francisco.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 1912, 'total_tokens': 1949, 'completion_time': 0.030833333, 'prompt_time': 0.314415932, 'queue_time': -0.370341837, 'total_time': 0.345249265}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-c7226a84-c072-4029-9313-049baa6c7c2c-0', usage_metadata={'input_tokens': 1912, 'output_tokens': 37, 'total_tokens': 1949})]

## **Streaming Messages**

In [39]:
for step in agent_executor.stream(
    {"messages":[HumanMessage(content="Use the TavilySearchResults tool to find out what the weather is like in San Francisco.")]},
    stream_mode='values',
):
    step['messages'][-1].pretty_print()

================================ Human Message =================================

Use the TavilySearchResults tool to find out what the weather is like in San Francisco.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_qy0a)
 Call ID: call_qy0a
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741417033, 'localtime': '2025-03-07 22:57'}, 'current': {'last_updated_epoch': 1741416300, 'last_updated': '2025-03-07 22:45', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/

## **Streaming tokens**

In [41]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="Use the TavilySearchResults tool to find out what the weather is like in San Francisco.")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco| is| clear| with| a| temperature| of| |52|°F| (|11|.|1|°C|).| The| wind| is| blowing| at| a| speed| of| |8|.|6| km|/h| (|5|.|4| mph|)| from| the| W|NW|.| The| humidity| is| |74|%| and| the| visibility| is| |9| miles| (|14|.|5| km|).| It| is| currently| |22|:|45|,| and| the| last| updated| weather| data| is| from| |22|:|45|.|

## **Adding in memory**

In [42]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [43]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [44]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 1905, 'total_tokens': 1909, 'completion_time': 0.003333333, 'prompt_time': 0.241674644, 'queue_time': -0.292026981, 'total_time': 0.245007977}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-92788495-d455-4c45-ae0c-de1749327d22-0', usage_metadata={'input_tokens': 1905, 'output_tokens': 4, 'total_tokens': 1909})]}}
----


In [45]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 1942, 'total_tokens': 1945, 'completion_time': 0.0025, 'prompt_time': 0.232802617, 'queue_time': -0.286801093, 'total_time': 0.235302617}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-665cf36f-98d7-4ff9-ac2a-a29d0c155fff-0', usage_metadata={'input_tokens': 1942, 'output_tokens': 3, 'total_tokens': 1945})]}}
----


In [46]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm not sure what your name is, as I don't have that information.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1909, 'total_tokens': 1927, 'completion_time': 0.015, 'prompt_time': 0.24790787, 'queue_time': -0.304519675, 'total_time': 0.26290787}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-97fe6c13-cbdb-4375-a9d4-9eafb17e68b9-0', usage_metadata={'input_tokens': 1909, 'output_tokens': 18, 'total_tokens': 1927})]}}
----
